# Coffea-Casa Benchmark Example 1

In [1]:
import numpy as np
import pytest
%matplotlib inline
from coffea import hist
import coffea.processor as processor
import awkward as ak

from dask.distributed import Client, LocalCluster
import time
import os
import ipytest

ipytest.config(rewrite_asserts=True, magics=True)

In [2]:
client = Client("tls://localhost:8786")
#cluster = LocalCluster()
#client = Client(cluster) 

fileset = {'SingleMu' : ["root://eospublic.cern.ch//eos/root-eos/benchmark/Run2012B_SingleMu.root"]}

from dask.distributed import Client, Worker, WorkerPlugin
from typing import List
import os

class DependencyInstaller(WorkerPlugin):
    def __init__(self, dependencies: List[str]):
        self._depencendies = " ".join(f"'{dep}'" for dep in dependencies)

    def setup(self, worker: Worker):
        os.system(f"pip install {self._depencendies}")


dependency_installer = DependencyInstaller([
    "pytest-benchmark",
])

client = Client("tls://localhost:8786")
client.register_worker_plugin(dependency_installer)

{'tls://oksana-2eshadura-40cern-2ech.dask-worker.coffea.casa:8788': {'status': 'OK'},
 'tls://red-c5423.unl.edu:49193': {'status': 'OK'},
 'tls://red-c5605.unl.edu:49160': {'status': 'OK'},
 'tls://red-c5609.unl.edu:49186': {'status': 'OK'},
 'tls://red-c5617.unl.edu:49183': {'status': 'OK'},
 'tls://red-c6909.unl.edu:49153': {'status': 'OK'}}

In [3]:
# This program plots an event-level variable (in this case, MET, but switching it is as easy as a dict-key change). It also demonstrates an easy use of the book-keeping cutflow tool, to keep track of the number of events processed.

# The processor class bundles our data analysis together while giving us some helpful tools.  It also leaves looping and chunks to the framework instead of us.
class Processor(processor.ProcessorABC):
    def __init__(self):
        # Bins and categories for the histogram are defined here. For format, see https://coffeateam.github.io/coffea/stubs/coffea.hist.hist_tools.Hist.html && https://coffeateam.github.io/coffea/stubs/coffea.hist.hist_tools.Bin.html
        dataset_axis = hist.Cat("dataset", "")
        MET_axis = hist.Bin("MET", "MET [GeV]", 50, 0, 100)
        
        # The accumulator keeps our data chunks together for histogramming. It also gives us cutflow, which can be used to keep track of data.
        self._accumulator = processor.dict_accumulator({
            'MET': hist.Hist("Counts", dataset_axis, MET_axis),
            'cutflow': processor.defaultdict_accumulator(int)
        })
    
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, events):
        output = self.accumulator.identity()
        
        # This is where we do our actual analysis. The dataset has columns similar to the TTree's; events.columns can tell you them, or events.[object].columns for deeper depth.
        dataset = events.metadata["dataset"]
        MET = events.MET.pt
        
        # We can define a new key for cutflow (in this case 'all events'). Then we can put values into it. We need += because it's per-chunk (demonstrated below)
        output['cutflow']['all events'] += ak.size(MET)
        output['cutflow']['number of chunks'] += 1
        
        # This fills our histogram once our data is collected. The hist key ('MET=') will be defined in the bin in __init__.
        output['MET'].fill(dataset=dataset, MET=MET)
        return output

    def postprocess(self, accumulator):
        return accumulator

In [4]:
def coffea_processor_adlexample1(chunk_size):
  output = processor.run_uproot_job(fileset,
                                  treename = 'Events',
                                  processor_instance = Processor(),
                                  executor = processor.dask_executor,
                                  chunksize = chunk_size,
                                  executor_args = {'schema': processor.NanoAODSchema, 'client': client}
                                )

In [5]:
# Generates a 1D histogram from the data output to the 'MET' key. fill_opts are optional, to fill the graph (default is a line).
def coffea_plot_adlexample1():
    hist.plot1d(output['MET'], overlay='dataset', fill_opts={'edgecolor': (0,0,0,0.3), 'alpha': 0.8})

In [6]:
  @pytest.mark.parametrize("chunk_size", range(100000, 500000, 100000))
  def test_coffea_processor_adlexample1(benchmark, chunk_size):
        benchmark(coffea_processor_adlexample1, chunk_size)
        coffea_plot_adlexample1()

In [7]:
ipytest.run("-qq")


------------------------- CSV report: coffea-bench.csv -------------------------

--------------------------------------------------------- benchmark: 1 tests ---------------------------------------------------------
Name (time in s)                                  Min      Max     Mean  StdDev   Median     IQR  Outliers     OPS  Rounds  Iterations
--------------------------------------------------------------------------------------------------------------------------------------
test_coffea_processor_adlexample1[100000]     57.7576  69.9881  62.4777  5.3210  61.0825  7.2020       1;0  0.0160       4           1
--------------------------------------------------------------------------------------------------------------------------------------

Legend:
  Outliers: 1 Standard Deviation from Mean; 1.5 IQR (InterQuartile Range) from 1st Quartile and 3rd Quartile.
  OPS: Operations Per Second, computed as 1 / Mean
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! KeyboardInterrupt !!!!!!!!!!!!!!!!!!!!!!!